In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import datetime
import json

In [2]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///./Data/COVID_Data.db")

In [3]:
#Verify that the database is connected and get table names for reference
inspector = inspect(engine)
inspector.get_table_names()

['usa_data', 'world_data']

In [4]:
#Verify column names for queries and future use
columns = inspector.get_columns('usa_data')
column_names = []
for c in columns:
    column_names.append(c['name'])
    print(c['name'], c["type"])
column_names

submission_date DATE
state TEXT
tot_cases INTEGER
conf_cases INTEGER
prob_cases INTEGER
new_case INTEGER
pnew_case INTEGER
tot_death INTEGER
conf_death INTEGER
prob_death INTEGER
new_death INTEGER
pnew_death INTEGER
created_at DATETIME
consent_cases TEXT
consent_deaths TEXT


['submission_date',
 'state',
 'tot_cases',
 'conf_cases',
 'prob_cases',
 'new_case',
 'pnew_case',
 'tot_death',
 'conf_death',
 'prob_death',
 'new_death',
 'pnew_death',
 'created_at',
 'consent_cases',
 'consent_deaths']

In [5]:
#get the parameters to run test (not used in final product)
states = ["AL","AK","AR", "AS","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
date_start = input("Start Date MM/DD/YYYY: ")
date_start_dt = datetime.datetime.strptime(date_start, '%m/%d/%Y').date()
date_end = input("End Date MM/DD/YYYY: ")
date_end_dt = datetime.datetime.strptime(date_start, '%m/%d/%Y').date()
state_select = input("State ID: ")
print(type(date_start))

Start Date MM/DD/YYYY: 12/21/2020
End Date MM/DD/YYYY: 12/23/2020
State ID: AL
<class 'str'>


In [6]:
#create placeholders for data
state_cases = []
state_new = []
state_tot_deaths = []
state_new_death = []

In [7]:
#run queries for data
#total cases by day
state_cases_query = engine.execute(f"SELECT tot_cases FROM usa_data WHERE state = '{state_select}' AND submission_date BETWEEN '{date_start}' AND '{date_end}'").fetchall()
for state in state_cases_query:
    state_cases.append(state[0])
state_cases

[326316, 860840, 334878, 330904, 863011]

In [8]:
#new cases by day
state_newcases_query = engine.execute(f"SELECT new_case FROM usa_data WHERE state = '{state_select}' AND submission_date BETWEEN '{date_start}' AND '{date_end}'").fetchall()
for state in state_newcases_query:
    state_new.append(state[0])
state_new

[4918, 1630, 3974, 4588, 2171]

In [9]:
#total deaths by day
state_totdeaths_query = engine.execute(f"SELECT tot_death FROM usa_data WHERE state = '{state_select}' AND submission_date BETWEEN '{date_start}' AND '{date_end}'").fetchall()
for state in state_totdeaths_query:
    state_tot_deaths.append(state[0])
state_tot_deaths

[6493, 16617, 6639, 6559, 16625]

In [10]:
#new deaths by day
state_newdeaths_query = engine.execute(f"SELECT new_death FROM usa_data WHERE state = '{state_select}' AND submission_date BETWEEN '{date_start}' AND '{date_end}'").fetchall()
for state in state_newdeaths_query:
    state_new_death.append(state[0])
state_new_death

[67, 8, 80, 66, 8]

In [15]:
#take lists and create dictionary for JSON output
i = 0
curr_date = date_start_dt - datetime.timedelta(days = 1)
output = []
#match the day data into a dictionary
while i < len(state_cases):
    #get the date
    curr_date = curr_date + datetime.timedelta(days = 1)
    str_date  = str(curr_date)
    print(curr_date)
    #build the dictionary
    data_dict = {
        'date': str_date,
        'total_cases': state_cases[i],
        'new_cases': state_new[i],
        'total_deaths': state_tot_deaths[i],
        'new_deaths': state_new_death[i]
    }
    #append to file
    output.append(data_dict)
    i=i+1
output

2020-12-21
2020-12-22
2020-12-23
2020-12-24
2020-12-25


[{'date': '2020-12-21',
  'total_cases': 326316,
  'new_cases': 4918,
  'total_deaths': 6493,
  'new_deaths': 67},
 {'date': '2020-12-22',
  'total_cases': 860840,
  'new_cases': 1630,
  'total_deaths': 16617,
  'new_deaths': 8},
 {'date': '2020-12-23',
  'total_cases': 334878,
  'new_cases': 3974,
  'total_deaths': 6639,
  'new_deaths': 80},
 {'date': '2020-12-24',
  'total_cases': 330904,
  'new_cases': 4588,
  'total_deaths': 6559,
  'new_deaths': 66},
 {'date': '2020-12-25',
  'total_cases': 863011,
  'new_cases': 2171,
  'total_deaths': 16625,
  'new_deaths': 8}]

In [16]:
#jsonify and send out
json_string = json.dumps(output)
print(json_string)

[{"date": "2020-12-21", "total_cases": 326316, "new_cases": 4918, "total_deaths": 6493, "new_deaths": 67}, {"date": "2020-12-22", "total_cases": 860840, "new_cases": 1630, "total_deaths": 16617, "new_deaths": 8}, {"date": "2020-12-23", "total_cases": 334878, "new_cases": 3974, "total_deaths": 6639, "new_deaths": 80}, {"date": "2020-12-24", "total_cases": 330904, "new_cases": 4588, "total_deaths": 6559, "new_deaths": 66}, {"date": "2020-12-25", "total_cases": 863011, "new_cases": 2171, "total_deaths": 16625, "new_deaths": 8}]


In [ ]:
#send json to file